In [1]:
import numpy as np
import pandas as pd
from rdkit import Chem, RDLogger
from rdkit.Chem import Descriptors, Lipinski, Crippen, rdMolDescriptors, AllChem

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

## Read data, perform some general cleaning and set up small molecule dataset

In [2]:
df1 = pd.read_csv('data/sm_data.csv', sep='";"', engine='python')
df1.rename({'"ChEMBL ID':'ChEMBL ID'}, axis=1, inplace=True)
df1.rename({'Inchi Key"':'InChI Key'}, axis=1, inplace=True)
df1.rename({'Smiles':'SMILES'}, axis=1, inplace=True)

df1['ChEMBL ID'] = df1['ChEMBL ID'].str[1:]
df1['InChI Key'] = df1['InChI Key'].str[:-1]

In [3]:
df1.columns

Index(['ChEMBL ID', 'Name', 'Synonyms', 'Type', 'Max Phase',
       'Molecular Weight', 'Targets', 'Bioactivities', 'AlogP',
       'Polar Surface Area', 'HBA', 'HBD', '#RO5 Violations',
       '#Rotatable Bonds', 'Passes Ro3', 'QED Weighted', 'CX Acidic pKa',
       'CX Basic pKa', 'CX LogP', 'CX LogD', 'Aromatic Rings',
       'Structure Type', 'Inorganic Flag', 'Heavy Atoms', 'HBA (Lipinski)',
       'HBD (Lipinski)', '#RO5 Violations (Lipinski)',
       'Molecular Weight (Monoisotopic)', 'Np Likeness Score',
       'Molecular Species', 'Molecular Formula', 'SMILES', 'InChI Key'],
      dtype='object')

In [4]:
#select important columns for data processing and analysis
subset1 = ['ChEMBL ID', 'InChI Key', 'Name', 'SMILES', 'Type', 'Max Phase', 'Targets', 'Bioactivities']

df2 = df1[subset1]
df2 = df2[df2['SMILES'].notna()]
df2.reset_index(drop=True, inplace=True)
df2[['Targets','Bioactivities']] = df2[['Targets','Bioactivities']].fillna(0)

In [5]:
df2['Type'].unique()

array(['Small molecule', 'Protein', 'Oligosaccharide', 'Unknown',
       'Oligonucleotide', 'Antibody'], dtype=object)

In [6]:
#select small molecules only and reset index
df3 = df2.loc[df2['Type']=='Small molecule']
df3.reset_index(drop=True, inplace=True)

## Calculate molecular features based on SMILES code

In [7]:
#function that calculates molecular features based on a SMILES line code
def features(smiles, verbose=False):
    
    #disable warnings, use with caution
    RDLogger.DisableLog('rdApp.*')
    
    try:
        molecule_data = []
        
        for element in smiles:
            mol = Chem.MolFromSmiles(element)
            molecule_data.append(mol)

        baseData = np.arange(1,1)
        i = 0  
        for mol in molecule_data:        

            parameter_MolWt = Descriptors.MolWt(mol)
            parameter_MolLogP = Descriptors.MolLogP(mol)
            parameter_MolMR = Crippen.MolMR(mol)
            parameter_NumHDonors = Lipinski.NumHDonors(mol)
            parameter_NumHAcceptors = Lipinski.NumHAcceptors(mol)
            parameter_TPSA = rdMolDescriptors.CalcTPSA(mol)
            parameter_HeavyAtomCount = Lipinski.HeavyAtomCount(mol)
            parameter_NumAromaticRings = Lipinski.NumAromaticRings(mol)
            parameter_NumRotatableBonds = Lipinski.NumRotatableBonds(mol)
            parameter_RingCount = Lipinski.RingCount(mol)
            
            row = np.array([parameter_MolWt,
                            parameter_MolLogP,
                            parameter_MolMR,
                            parameter_NumHDonors,
                            parameter_NumHAcceptors,
                            parameter_TPSA,
                            parameter_HeavyAtomCount,
                            parameter_NumAromaticRings, 
                            parameter_NumRotatableBonds,
                            parameter_RingCount])   

            if(i==0):
                baseData=row
            else:
                baseData=np.vstack([baseData, row])
            i = i+1      

        columnNames = ['MW','LogP','MolMR','H Donors','H Acceptors', 'TPSA',
                     'Heavy Atom Count', 'Aromatic Rings', 'Rotatable Bonds', 'Ring Count']   
        parameters = pd.DataFrame(data=baseData, columns=columnNames)
        
    except:
        print('Check for NaN values in the smiles column')

    return parameters

In [8]:
#function that calculates the Morgan fingerprint based on a SMILES line code    
def morgan_fp(df, radius=3, nBits=1024):
    
    #disable warnings, use with caution
    RDLogger.DisableLog('rdApp.*')
    
    # create an empty dataframe with column names
    df_morgan = pd.DataFrame(columns=[f'Bit_{i}' for i in range(nBits)])  

    for i, smiles in enumerate(df['SMILES']):
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            continue

        #generate Morgan fingerprints and add to df_morgan
        fingerprint = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=nBits)
        fp_bits = list(fingerprint.ToBitString())
        df_morgan.loc[i] = fp_bits  # add the fingerprint bits as a row to the dataframe

    # set the index to be the SMILES code and add a name for the index column
    df_morgan.index = df3['SMILES']  
    df_morgan.index.name = 'SMILES' 
    
    return df_morgan

In [9]:
#run features() function on SMILES codes
df_features = features(df3.SMILES)
df_features.dropna(inplace=True)

#run morgan_fp() function on SMILES codes
df_morgan = morgan_fp(df3, radius=3, nBits=1024)
df_morgan.reset_index(inplace=True, drop=True)

#combine the calculated data with the existing data sets
df4 = pd.concat([df3, df_features, df_morgan], axis=1)
df4

ChEMBL ID                    InChI Key                         Name  \
0     CHEMBL3039533  RXSARIJMSJWJLZ-CIAYNJNFSA-N                  GRAZOPREVIR   
1     CHEMBL1201060  JSXBVMKACNEMKY-UHFFFAOYSA-N       LOXAPINE HYDROCHLORIDE   
2     CHEMBL2096648  XSQUKJJJFZCRTK-OUBTZVSYSA-N                    UREA C 13   
3     CHEMBL2219411  IPNATXQRPWRHKD-UHFFFAOYSA-N  MOMELOTINIB DIHYDROCHLORIDE   
4     CHEMBL1200366  YHJDZIQOCSDIQU-OEDJVVDHSA-N     CEPHALEXIN HYDROCHLORIDE   
...             ...                          ...                          ...   
6804  CHEMBL4650320  HTNPEHXGEKVIHG-QCNRFFRDSA-N                 MOLNUPIRAVIR   
6805  CHEMBL4650322  NCRMKIWHFXSBGZ-CNBXIYLPSA-N                  RUNCACIGUAT   
6806      CHEMBL103  RJKFOVLPORLFTN-LEKSSAKUSA-N                 PROGESTERONE   
6807    CHEMBL13817  UMUPQWIGCOZEOY-JOCHJYFZSA-N                   IBUTAMOREN   
6808    CHEMBL13828  BAINIUMDFURPJM-UHFFFAOYSA-N                    OXATOMIDE   

                                                 SMILES            Type  \
0     C=C[C@@H]1C[C@]1(NC(=O)[C@@H]1C[C@@H]2CN1C(=O)...  Small molecule   
1              CN1CCN(C2=Nc3ccccc3Oc3ccc(Cl)cc32)CC1.Cl  Small molecule   
2                                           N[13C](N)=O  Small molecule   
3     Cl.Cl.N#CCNC(=O)c1ccc(-c2ccnc(Nc3ccc(N4CCOCC4)...  Small molecule   
4     CC1=C(C(=O)O)N2C(=O)[C@@H](NC(=O)[C@H](N)c3ccc...  Small molecule   
...                                                 ...             ...   
6804  CC(C)C(=O)OC[C@H]1O[C@@H](n2cc/c(=N\O)[nH]c2=O...  Small molecule   
6805  C[C@H]([C@H](C(=O)Nc1cc([C@@H](CC(=O)O)C2CC2)c...  Small molecule   
6806  CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(...  Small molecule   
6807  CC(C)(N)C(=O)N[C@H](COCc1ccccc1)C(=O)N1CCC2(CC...  Small molecule   
6808  O=c1[nH]c2ccccc2n1CCCN1CCN(C(c2ccccc2)c2ccccc2...  Small molecule   

      Max Phase  Targets  Bioactivities          MW     LogP     MolMR  \
0           3.0      0.0            0.0  784.933000  2.47720  200.4267   
1           4.0      0.0            0.0  364.276000  4.19320  100.4830   
2           4.0      0.0            0.0   61.048355 -0.97620   13.7708   
3           1.0      0.0            0.0  487.391000  3.82078  132.6669   
4           4.0      0.0            0.0  401.872000  0.04200   99.5907   
...         ...      ...            ...         ...      ...       ...   
6804        3.0      1.0            4.0  329.309000 -1.71530   74.1228   
6805        2.0      9.0           42.0  488.333000  6.88240  117.2115   
6806        4.0    286.0         1493.0  314.469000  4.72350   90.8810   
6807        2.0     12.0           72.0  528.675000  1.76530  142.4189   
6808        4.0     55.0          180.0  426.564000  4.12690  129.9157   

      H Donors  H Acceptors    TPSA  Heavy Atom Count  Aromatic Rings  \
0          3.0         11.0  226.72              55.0             2.0   
1          0.0          4.0   28.07              24.0             2.0   
2          2.0          1.0   69.11               4.0             0.0   
3          2.0          7.0  103.17              33.0             3.0   
4          3.0          5.0  144.23              26.0             1.0   
...        ...          ...     ...               ...             ...   
6804       4.0          9.0  146.37              23.0             1.0   
6805       2.0          2.0   66.40              32.0             2.0   
6806       0.0          2.0   34.14              23.0             0.0   
6807       2.0          6.0  122.04              37.0             2.0   
6808       1.0          4.0   44.27              32.0             4.0   

      Rotatable Bonds  Ring Count Bit_0 Bit_1 Bit_2 Bit_3 Bit_4 Bit_5 Bit_6  \
0                 7.0         7.0     0     0     1     0     1     1     0   
1                 0.0         4.0     0     0     0     0     0     0     0   
2                 0.0         0.0     0     0     0     0     0     0     0   
3                 6.0         4.0     0     0     0   

In [10]:
#save dataset
df4.to_csv('data/sm_data_features_extensive.csv', index=False)